In [1]:
import string

import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

nltk.download("stopwords")
import warnings

warnings.filterwarnings("ignore")
import re

from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brandon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
df = pd.read_csv("../data/raw/componded.csv")
df

,class,text
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
248327,2,""":::::And for the second time of asking, when ..."
248328,2,You should be ashamed of yourself \n\nThat is ...
248329,2,"Spitzer \n\nUmm, theres no actual article for ..."
248330,2,And it looks like it was actually you who put ...


In [20]:
df["text"].dropna(inplace=True)

### Dictionary of Contractions

In [21]:
cList = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you you will",
    "you'll've": "you you will have",
    "you're": "you are",
    "you've": "you have",
}

In [22]:
c_re = re.compile("(%s)" % "|".join(cList.keys()))


def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]

    return c_re.sub(replace, text)


lst = []
for text in tqdm(df["text"]):
    lst.append(expandContractions(text))

df["text"] = lst

100%|██████████| 248332/248332 [00:17<00:00, 14162.31it/s]


In [23]:
# Remove punctuation
df["text"] = df["text"].str.replace("[{}]".format(string.punctuation), "")

In [24]:
# Remove newlines
df["text"] = df["text"].str.replace("[{}]".format("\n"), "")

In [25]:
# Remove numbers
df["text"] = df["text"].str.replace("[{}]".format(r"\d+"), "")

In [26]:
# Remove links
df.replace({"text": r"http\w*"}, {"text": ""}, regex=True, inplace=True)

In [27]:
# Remove trailing whitespace
df["text"] = df["text"].str.strip()

In [28]:
# Remove duplicate spaces
df["text"] = df["text"].str.replace(r" {2,}", " ")

In [29]:
# Save processed text
df.to_csv("../data/processed/processed.csv", index=False)

In [30]:
# Remove stopwords
pattern = r"\b(?:{})\b".format("|".join( ))
df["text"] = df["text"].str.replace(pattern, "")

In [31]:
df["text"] = df["text"].str.replace(r" {2,}", " ")

In [32]:
# Save processed text with stopwords removed
df.to_csv(
    "../data/processed/processed_stopwords.csv", index=False,
)